"""
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
"""
"""
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
       
"""
 """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
"""

## POLICY EVALUATION

In [1]:
import numpy as np 
import gym.spaces
from gridworld import GridworldEnv
env = GridworldEnv()

def policy_eval(policy, env, discount_factor=1.0, epsilon=0.00001, maxiter = 10000):
    V_old = np.zeros(env.nS)
    for i in range(maxiter): # or while True:
        delta = 0
        for s in range(env.nS):
            v_fn = 0
            action_probs = policy[s]
            for a in range(env.nA):
                [(prob, next_state, reward, done)] = env.P[s][a]
                v_fn += action_probs[a] * (reward + discount_factor * V_old[next_state])
            delta = max(delta, abs(v_fn - V_old[s]))
            V_old[s] = v_fn
        if delta < epsilon:
            break
    return np.array(V_old)

random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)
print(v)
print(expected_v)


[  0.         -13.99993529 -19.99990698 -21.99989761 -13.99993529
 -17.9999206  -19.99991379 -19.99991477 -19.99990698 -19.99991379
 -17.99992725 -13.99994569 -21.99989761 -19.99991477 -13.99994569
   0.        ]
[  0 -14 -20 -22 -14 -18 -20 -20 -20 -20 -18 -14 -22 -20 -14   0]


## POLICY ITERATION

In [2]:
import numpy as np
import gym.spaces
from gridworld import GridworldEnv
env = GridworldEnv()

def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    def one_step_lookahead(s, value_fn):
        actions = np.zeros(env.nA)
        for a in range(env.nA):
            [(prob, next_state, reward, done)] = env.P[s][a]
            actions[a] = prob * (reward + discount_factor * value_fn[next_state]) 
        return actions

    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    actions_values = np.zeros(env.nA)

    while True:
        value_fn = policy_eval_fn(policy, env)
        policy_stable = True
        for s in range(env.nS):
            actions_values = one_step_lookahead(s, value_fn) 
            best_action = np.argmax(actions_values) #best action over one step look ahead..
            chosen_action = np.argmax(policy[s]) #best action on current policy
            if(best_action != chosen_action):
                policy_stable = False
            policy[s] = np.eye(env.nA)[best_action]#the updated new policy after acting greedily w.r.t value function
            # [0,1,2,3]-env.na, say best action-2, np.eye will make [0,0,1,0]
        if(policy_stable):
            return policy, value_fn

policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



# VALUE ITERATION

In [3]:
import numpy as np
from gridworld import GridworldEnv
env = GridworldEnv()

def value_iteration(env, epsilon=0.0001, discount_factor=1.0):
    def one_step_lookahead(s, value_fn):
        actions = np.zeros(env.nA)
        for a in range(env.nA):
            [(prob, next_state, reward, done)] = env.P[s][a]
            actions[a] = prob * (reward + discount_factor * value_fn[next_state])
        return actions
    
    value_fn = np.zeros(env.nS)
    policy = np.zeros([env.nS, env.nA])
    while True:
    	delta = 0
    	for s in range(env.nS):
    		actions_values = one_step_lookahead(s, value_fn)
    		best_action_value = max(actions_values)
    		delta = max(delta, abs(best_action_value - value_fn[s]))
    		value_fn[s] = best_action_value #apply bellman optimality eqn
    		best_action = np.argmax(actions_values)#to update the policy
    		policy[s] = np.eye(env.nA)[best_action]#to update the policy
    	if(delta < epsilon):
    		break
    return policy, value_fn

policy, v = value_iteration(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")


Policy Probability Distribution:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



## check time taken!!!

In [4]:
%%time
policy, v = policy_improvement(env)

Wall time: 12 ms


In [5]:
%%time
policy, v = value_iteration(env)

Wall time: 1.5 ms
